# 0. 노드, 연결리스트

In [1]:
class Node:

    def __init__(self, item):
        self.data = item
        self.prev = None
        self.next = None


class DoublyLinkedList:

    def __init__(self):
        self.nodeCount = 0
        self.head = Node(None)
        self.tail = Node(None)
        self.head.prev = None
        self.head.next = self.tail
        self.tail.prev = self.head
        self.tail.next = None


    def __repr__(self):
        if self.nodeCount == 0:
            return 'LinkedList: empty'

        s = ''
        curr = self.head
        while curr.next.next:
            curr = curr.next
            s += repr(curr.data)
            if curr.next.next is not None:
                s += ' -> '
        return s


    def getLength(self):
        return self.nodeCount


    def traverse(self):
        result = []
        curr = self.head
        while curr.next.next:
            curr = curr.next
            result.append(curr.data)
        return result


    def reverse(self):
        result = []
        curr = self.tail
        while curr.prev.prev:
            curr = curr.prev
            result.append(curr.data)
        return result


    def getAt(self, pos):
        if pos < 0 or pos > self.nodeCount:
            return None

        if pos > self.nodeCount // 2:
            i = 0
            curr = self.tail
            while i < self.nodeCount - pos + 1:
                curr = curr.prev
                i += 1
        else:
            i = 0
            curr = self.head
            while i < pos:
                curr = curr.next
                i += 1

        return curr


    def insertAfter(self, prev, newNode):
        next = prev.next
        newNode.prev = prev
        newNode.next = next
        prev.next = newNode
        next.prev = newNode
        self.nodeCount += 1
        return True


    def insertAt(self, pos, newNode):
        if pos < 1 or pos > self.nodeCount + 1:
            return False

        prev = self.getAt(pos - 1)
        return self.insertAfter(prev, newNode)


    def popAfter(self, prev):
        curr = prev.next
        next = curr.next
        prev.next = next
        next.prev = prev
        self.nodeCount -= 1
        return curr.data


    def popAt(self, pos):
        if pos < 1 or pos > self.nodeCount:
            return None

        prev = self.getAt(pos - 1)
        return self.popAfter(prev)


    def concat(self, L):
        self.tail.prev.next = L.head.next
        L.head.next.prev = self.tail.prev
        self.tail = L.tail

        self.nodeCount += L.nodeCount

# 1. 스택(Stack)

In [ ]:
class ArrayStack:

    def __init__(self):
        self.data = []

    def size(self):
        return len(self.data)

    def isEmpty(self):
        return self.size() == 0

    def push(self, item):
        self.data.append(item)

    def pop(self):
        return self.data.pop()

    def peek(self):
        return self.data[-1]

### 1-2 중위 표현식 -> 후위 표현식 변환 (def infixToPostfix(tokenList))
### 1-3 후위 표현식 계산 (def postfixEval(tokenList))

In [4]:
def splitTokens(exprStr):
    tokens = []
    val = 0
    valProcessing = False
    for c in exprStr:
        if c == ' ':
            continue
        if c in '0123456789':
            val = val * 10 + int(c)
            valProcessing = True
        else:
            if valProcessing:
                tokens.append(val)
                val = 0
            valProcessing = False
            tokens.append(c)
    if valProcessing:
        tokens.append(val)

    return tokens


def infixToPostfix(tokenList):
    prec = {
        '*': 3,
        '/': 3,
        '+': 2,
        '-': 2,
        '(': 1,
    }

    opStack = ArrayStack()
    postfixList = []
    for token in tokenList:
        if token=='(':
            opStack.push(token)
        elif token==')':
            while opStack.peek()!='(':
                postfixList.append(opStack.pop())
            opStack.pop()
        elif token not in prec.keys():
            postfixList.append(token)
        else:
            if opStack.isEmpty():
                opStack.push(token)
            else:
                while opStack.isEmpty()!=True and prec[opStack.peek()]>=prec[token]:
                    postfixList.append(opStack.pop())
                opStack.push(token)
    while opStack.isEmpty()!=True:
        if opStack not in ['(', ')']:
            postfixList.append(opStack.pop())
        else:
            opStack.pop()
    return postfixList


def postfixEval(tokenList):
    valStack = ArrayStack()
    
    for token in tokenList:
        if type(token) is int:
            valStack.push(token)
        elif token == '*':
            num2 = valStack.pop()
            num1 = valStack.pop()
            valStack.push(num1 * num2)
        elif token == '/':
            num2 = valStack.pop()
            num1 = valStack.pop()
            valStack.push(num1 / num2)
        elif token == '+':
            num2 = valStack.pop()
            num1 = valStack.pop()
            valStack.push(num1 + num2)
        elif token == '-':
            num2 = valStack.pop()
            num1 = valStack.pop()
            valStack.push(num1 - num2)
    
    return valStack.pop()


def solution(expr):
    tokens = splitTokens(expr)
    postfix = infixToPostfix(tokens)
    val = postfixEval(postfix)
    return val

# 2. 큐(Queue)

### 1-1. 큐 - 연결리스트로 구현

In [3]:
class LinkedListQueue:

    def __init__(self):
        self.data = DoublyLinkedList()

    def size(self):
        return self.data.getLength()


    def isEmpty(self):
        return self.data.getLength()==0


    def enqueue(self, item):
        node = Node(item)     
        self.data.insertAt(self.size()+1, node)


    def dequeue(self):
        return self.data.popAt(1)


    def peek(self):
        return self.data.getAt(1).data


### 2-2 원형 큐(Circular Queue)

In [ ]:
class CircularQueue:

    def __init__(self, n):
        self.maxCount = n
        self.data = [None] * n
        self.count = 0
        self.front = -1
        self.rear = -1


    def size(self):
        return self.count

    def isEmpty(self):
        return self.count == 0

    def isFull(self):
        return self.count == self.maxCount

    def enqueue(self, x):
        if self.isFull():
            raise IndexError('Queue full')
        self.rear = (self.rear+1)%self.maxCount

        self.data[self.rear] = x
        self.count += 1

    def dequeue(self):
        if self.size()==0:
            raise IndexError('Queue empty')
        self.front = (self.front+1)%self.maxCount

        x = self.data[self.front]

        self.count -= 1
        return x

    def peek(self):
        if self.isEmpty():
            raise IndexError('Queue empty')
        return self.data[(self.front+1)%self.maxCount]


### 2-3. 우선순위 큐(Priority Queue)

In [ ]:
class PriorityQueue:

    def __init__(self):
        self.queue = DoublyLinkedList()


    def size(self):
        return self.queue.getLength()

    def isEmpty(self):
        return self.size() == 0

    def enqueue(self, x):
        newNode = Node(x)
        curr = self.queue.head

        while curr.next.data!=None and curr.next.data>x:
            curr = curr.next
        self.queue.insertAfter(curr, newNode)

    def dequeue(self):
        return self.queue.popAt(self.queue.getLength())

    def peek(self):
        return self.queue.getAt(self.queue.getLength()).data


def solution(x):
    return 0

# 3. 이진트리

### 함수 구현
- size()
- depth()

In [ ]:
class Node:

    def __init__(self, item):
        self.data = item
        self.left = None
        self.right = None


    def size(self):
        l = self.left.size() if self.left else 0
        r = self.right.size() if self.right else 0
        return l + r + 1


    def depth(self):
        l=self.left.depth() if self.left else 0
        r=self.right.depth() if self.right else 0
        return max(l, r)+1


class BinaryTree:

    def __init__(self, r):
        self.root = r

    def size(self):
        if self.root:
            return self.root.size()
        else:
            return 0


    def depth(self):
        if self.root:
            return self.root.depth()
        else:
            return 0
        


def solution(x):
    return 0

- traverse() 순회 구현 - 깊이 우선방식/넓이 우선 방식
1) 깊이 우선 방식
- inorder
- preorder
- postorder

In [ ]:
class Node:

    def __init__(self, item):
        self.data = item
        self.left = None
        self.right = None


    def inorder(self):
        traversal = []
        if self.left:
            traversal += self.left.inorder()
        traversal.append(self.data)
        if self.right:
            traversal += self.right.inorder()
        return traversal


    def preorder(self):
        traversal = []
        traversal.append(self.data)
        if self.left:
            traversal += self.left.preorder()
        if self.right:
            traversal += self.right.preorder()
        return traversal
    

    def postorder(self):
        traversal=[]
        if self.left:
            traversal+=self.left.postorder()
        if self.right:
            traversal+=self.right.postorder()
        traversal.append(self.data)
        return traversal


class BinaryTree:

    def __init__(self, r):
        self.root = r


    def inorder(self):
        if self.root:
            return self.root.inorder()
        else:
            return []


    def preorder(self):
        if self.root:
            return self.root.preorder()
        else:
            return []
        
    
    
    def postorder(self):
        if self.root:
            return self.root.postorder()
        else:
            return []


def solution(x):
    return 0

2) 넓이 우선 방식 - queue 이용

In [ ]:
class ArrayQueue:

    def __init__(self):
        self.data = []

    def size(self):
        return len(self.data)

    def isEmpty(self):
        return self.size() == 0

    def enqueue(self, item):
        self.data.append(item)

    def dequeue(self):
        return self.data.pop(0)

    def peek(self):
        return self.data[0]


class Node:

    def __init__(self, item):
        self.data = item
        self.left = None
        self.right = None


class BinaryTree:

    def __init__(self, r):
        self.root = r


    def bft(self):
        traverse=[]
        if self.root:
            q=ArrayQueue()
            q.enqueue(self.root)
            while q.isEmpty()==False:
                curr=q.dequeue()
                traverse.append(curr.data)
                if curr.left!=None:
                    q.enqueue(curr.left)
                if curr.right!=None:
                    q.enqueue(curr.right)
        else:
            return []
        return traverse


def solution(x):
    return 0

# 4. 이진탐색트리
- insert(key, data) : 트리에 key 자리에 데이터 원소를 추가
- lookup(key) : key에 해당하는 노드와 부모노드를 반환
- inorder() : key의 순서대로 데이터 원소를 나열
- remove(key) : 트리에 key 데이터 원소를 삭제
- min() : 최소키를 가지는 원소 탐색
- max() : 최대키를 가지는 원소 탐색

In [ ]:
class Node:

    def __init__(self, key, data):
        self.key = key
        self.data = data
        self.left = None
        self.right = None


    def insert(self, key, data):
        if key<self.key:
            if self.left:
                return self.left.insert(key, data)
            else:
                self.left=Node(key, data)
        
        elif key>self.key:
            if self.right:
                return self.right.insert(key, data)
            else:
                self.right=Node(key, data)
        else: #해당 노드에 삽입               
            raise KeyError
        
    
    def lookup(self, key, parent=None):
        if key < self.key:
            if self.left:
                return self.left.lookup(key, self)
            else:
                return None, None
        elif key > self.key:
            if self.right:
                return self.right.lookup(key, self)
            else:
                return None, None
        else:
            return self, parent


    def inorder(self):
        traversal = []
        if self.left:
            traversal += self.left.inorder()
        traversal.append(self)
        if self.right:
            traversal += self.right.inorder()
        return traversal
    

    def countChildren(self):
        count = 0
        if self.left:
            count += 1
        if self.right:
            count += 1
        return count


class BinSearchTree:

    def __init__(self):
        self.root = None


    def insert(self, key, data):
        if self.root:
            self.root.insert(key, data)
        else:
            self.root = Node(key, data)

    def lookup(self, key):
        if self.root:
            return self.root.lookup(key)
        else:
            return None, None


    def inorder(self):
        if self.root:
            return self.root.inorder()
        else:
            return []

    
    def remove(self, key):
        node, parent = self.lookup(key)
        if node:
            nChildren = node.countChildren()
            # The simplest case of no children
            if nChildren == 0:
                if parent:
                    if node==parent.left:
                        parent.left=None
                    else:
                        parent.right=None
                else:
                    self.root=None
            # When the node has only one child
            elif nChildren == 1:
                next=None
                if node.left:
                    next=node.left
                else:
                    next=node.right
                if parent:
                    if node==parent.left:
                        parent.left=next
                    else:
                        parent.right=next
                else:
                    self.root=next
            # When the node has both left and right children
            else:
                parent = node
                successor = node.right
                while successor.left:
                    parent=successor
                    successor=successor.left
                node.key = successor.key
                node.data = successor.data
                if parent.left==successor:
                    parent.left=successor.right
                else:
                    parent.right=successor.right
            return True

        else:
            return False


def solution(x):
    return 0